# coast

In [ ]:
# coast
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
import cartopy.crs as ccrs
from matplotlib import cm


file_path = os.path.expandvars('$WORK/data/OISST/monthly/sst.mon.mean.nc')
ds = xr.open_dataset(
    file_path
)

[xmin,xmax,ymin,ymax]=[330,355,10,35]
[tmin,tmax]=["1993","2022"]
sst=ds['sst'].sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax),time=slice(tmin,tmax)).values
sst0=ds['sst'].sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax)).isel(time=0).values

lon=ds['lon'].sel(lon=slice(xmin, xmax)).values
lat=ds['lat'].sel(lat=slice(ymin,ymax)).values
time=ds['time'].sel(time=slice(tmin,tmax))


    
coast = {'lat': [], 'lon': []}
    
for i, lat_val in enumerate(lat):
    for j, lon_val in enumerate(lon):            
        if np.isnan(sst0[i, j]):
            coast['lat'].append(lat_val.item())
            coast['lon'].append(lon_val.item())
            break  

# obs

## sstu

In [ ]:
# sstu
n = 16
sstc = np.empty((sst.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))


for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    sstc[:, point_idx, :] = sst[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
sstu=sstc[:,:,8:]-sstc[:,:,:8]
np.savez(f'sstu_{tmin}_{tmax}.npz', lon=lon, lat=lat, sstu=sstu, sstc=sstc, coast=coast, lonc=lonc, latc=latc, time=time)

## wind

In [ ]:
# wind ekman transport and pumping
import glob

data_dir = os.path.expandvars('$WORK/data/CCMP/monthly/')
file_pattern = os.path.join(data_dir, 'CCMP_Wind_Analysis_??????_monthly_mean_V03.1_L4.nc')
file_list = sorted(glob.glob(file_pattern))
ds = xr.open_mfdataset(file_list, combine='by_coords')
u=ds['u'].sel(longitude=slice(xmin,xmax), latitude=slice(ymin,ymax),time=slice(tmin,tmax)).values
v=ds['v'].sel(longitude=slice(xmin,xmax), latitude=slice(ymin,ymax),time=slice(tmin,tmax)).values

n = 8
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))
uc = np.empty((u.shape[0],len(coast['lat']), n))
vc = np.empty((v.shape[0],len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    uc[:, point_idx, :] = u[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    vc[:, point_idx, :] = v[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]

uc[np.abs(uc) > 100] = np.nan
vc[np.abs(vc) > 100] = np.nan

# alongshore wind
def calculate_bearing(lat1, lon1, lat2, lon2):
    lat1_rad = np.deg2rad(lat1)
    lat2_rad = np.deg2rad(lat2)
    lon1_rad = np.deg2rad(lon1)
    lon2_rad = np.deg2rad(lon2)
    
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    
    x = delta_lon * np.cos((lat1_rad + lat2_rad) / 2)
    y = delta_lat
    
    bearing_rad = np.arctan2(y, x)
    bearing_deg = np.rad2deg(bearing_rad)
    bearing_deg = (bearing_deg + 360) % 360
    
    return bearing_deg

point_num = len(coast['lat'])
coast_orientation = np.empty(point_num)

for i in range(point_num):
    if i < point_num - 1:
        lat1 = coast['lat'][i]
        lon1 = coast['lon'][i]
        lat2 = coast['lat'][i+1]
        lon2 = coast['lon'][i+1]
    else:
        lat1 = coast['lat'][i-1]
        lon1 = coast['lon'][i-1]
        lat2 = coast['lat'][i]
        lon2 = coast['lon'][i]
    
    bearing = calculate_bearing(lat1, lon1, lat2, lon2)
    coast_orientation[i] = bearing

coast_orientation_rad = np.deg2rad(coast_orientation)
u_alongshore = np.cos(coast_orientation_rad).reshape(1, point_num, 1)
v_alongshore = np.sin(coast_orientation_rad).reshape(1, point_num, 1)
wind_alongshore = uc * u_alongshore + vc * v_alongshore

# Define the haversine function
coast_lon = np.array(coast['lon'])  # Shape: (100,)
coast_lat = np.array(coast['lat'])  # Shape: (100,)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  

    # Convert degrees to radians
    lat1_rad = np.deg2rad(lat1)
    lon1_rad = np.deg2rad(lon1)
    lat2_rad = np.deg2rad(lat2)
    lon2_rad = np.deg2rad(lon2)

    # Compute differences
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    # Haversine formula
    a = np.sin(delta_lat / 2.0) ** 2 + \
        np.cos(lat1_rad) * np.cos(lat2_rad) * \
        np.sin(delta_lon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

# Expand coast_lon and coast_lat for broadcasting
coast_lon_expanded = coast_lon[:, np.newaxis]  # Shape: (100, 1)
coast_lat_expanded = coast_lat[:, np.newaxis]  # Shape: (100, 1)

# Compute distances
distances = haversine(coast_lat_expanded, coast_lon_expanded, latc, lonc)  # Shape: (100, 8)

# Constants
rhoa = 1.22  # air density (kg/m^3)
cd = 0.0013  # drag coefficient
rho = 1025  # water density (kg/m^3)
omega = 7.2921e-5  # Earth's rotation rate (rad/s)

f = 2 * omega * np.sin(latc* np.pi / 180)

# Calculate magnitude of horizontal velocity
windspeed = np.sqrt(uc**2 + vc**2)

# Ekman transport calculation
etransport = -rhoa * cd * windspeed * wind_alongshore / (rho * f * distances)


epumping = np.zeros_like(uc)  # Ensure it has the right shape and is mutable
dy = 111000.0 * (lat[1] - lat[0])  # scalar
dv_dx = np.zeros_like(uc)
for j in range(lat.shape[0]):
    x_j = 111000.0 * np.cos(np.radians(lat[j])) * (lon[1] - lon[0]) 
    epumping[:,j,:] = (rhoa * cd * windspeed[:,j,:] / (rho)) * (np.gradient(vc[:, j, :]/f[j], x_j, axis=-1) - np.gradient(uc[:,j,:]/f[j], dy, axis=1))

np.savez(f'ekman_{tmin}_{tmax}.npz', etransport=etransport, epumping=epumping)


## sla

In [ ]:
# sla
import glob

# Path template
data_dir = os.path.expandvars('$WORK/data/SSH/monthly/????/')
file_pattern = os.path.join(data_dir, 'dt_global_allsat_msla_h_y????_m??.nc')
file_list = sorted(glob.glob(file_pattern))
ds = xr.open_mfdataset(file_list, combine='by_coords')
sla=ds['sla'].sel(longitude=slice(xmin,xmax), latitude=slice(ymin,ymax),time=slice(tmin,tmax)).values

n = 8
slac = np.empty((sla.shape[0],len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    slac[:, point_idx, :] = sla[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
slacm=np.nanmean(slac,axis=0)  
slac -= slacm
np.savez(f'slac_{tmin}_{tmax}.npz', slac=slac)

## chl

In [ ]:
# chl
file_path = os.path.expandvars('$WORK/data/GlobColour/monthly/cmems_obs-oc_glo_bgc-plankton_my_l4-multi-4km_P1M_CHL_1997-09-01-2024-08-01.nc')
ds = xr.open_dataset(
    file_path
)
ds['longitude'] = (ds['longitude'] + 360) % 360
ds=ds.sortby('longitude')
CHL=ds['CHL'].sel(longitude=slice(xmin,xmax), latitude=slice(ymin,ymax),time=slice("1998",tmax))
CHL=CHL.interpolate_na(dim="longitude", method="nearest")
CHL=CHL.interpolate_na(dim="latitude", method="nearest")
CHL=CHL.bfill(dim="longitude").bfill(dim="latitude")
CHL=CHL.ffill(dim="longitude").ffill(dim="latitude")
lat_xr = xr.DataArray(lat, dims="latitude")
lon_xr = xr.DataArray(lon, dims="longitude")
chl = CHL.interp(latitude=lat_xr, longitude=lon_xr).values


n = 16
chlc = np.empty((chl.shape[0],len(coast['lat']), n))
for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    chlc[:, point_idx, :] = chl[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
np.savez(f'chlc_1998_{tmax}.npz', chlc=chlc)

In [ ]:
np.isnan(chlc).sum()

## mld

In [ ]:
# mld
import glob

data_dir = os.path.expandvars('$WORK/data/ARMOR/monthly/????/')
file_pattern = os.path.join(data_dir, 'dataset-armor-3d-rep-monthly_????????T????Z_P????????T????Z.nc')
file_list = sorted(glob.glob(file_pattern))
ds = xr.open_mfdataset(file_list, combine='by_coords')
ds['longitude'] = (ds['longitude'] + 360) % 360
ds=ds.sortby('longitude')
MLD=ds['mlotst'].sel(longitude=slice(xmin,xmax), latitude=slice(ymin,ymax),time=slice(tmin,tmax))
lat_xr = xr.DataArray(lat, dims="latitude")
lon_xr = xr.DataArray(lon, dims="longitude")
mld = MLD.interp(latitude=lat_xr, longitude=lon_xr).values

n = 16
mldc = np.empty((mld.shape[0],len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    mldc[:, point_idx, :] = mld[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
np.savez(f'mldc_{tmin}_{tmax}.npz', mldc=mldc)

## estimate ww

In [ ]:
# estimate ww
dy = 0.25 * 111320  # meters (constant)
dz = 70  # meters

def compute_ww(uu, vv, lats, dx_base=0.25 * 111320, dy=dy, dz=dz):
    # Convert latitude to radians
    lats_rad = np.radians(lats)
    
    # Compute dx for each latitude
    dx = dx_base * np.cos(lats_rad)  # meters

    # Expand dx to match (time, lon, lat) shape
    dx = dx[np.newaxis, np.newaxis, :]  # Shape (1, 1, lat)

    # Compute derivatives using central differences (ignoring edges)
    dudx = (uu[:, 2:, 1:-1] - uu[:, :-2, 1:-1]) / (2 * dx[:, :, 1:-1])
    dvdy = (vv[:, 1:-1, 2:] - vv[:, 1:-1, :-2]) / (2 * dy)

    # Compute ww using continuity equation
    ww = dz * (dudx + dvdy)

    # Pad the result to match original shape (setting edges to NaN)
    ww_full = np.full_like(uu, np.nan)
    ww_full[:, 1:-1, 1:-1] = ww

    return ww_full

In [ ]:
# OSCAR no interp
file_path = os.path.expandvars('$WORK/data/OSCAR/oscar_monthly_mean_1993_2022.nc')
ds = xr.open_dataset(
    file_path
)
ds = ds.where(
    (ds.lat >= ymin) & (ds.lat <= ymax) & 
    (ds.lon >= xmin) & (ds.lon <= xmax), 
    drop=True
)
u=ds['u']
v=ds['v']
ug=ds['ug']
vg=ds['vg']
u0=ds['u'].isel(time=0).interpolate_na(dim="longitude", method="linear")

uu = np.swapaxes(u.values, 1, 2)  # swapaxes to (time,lat,lon)
vv = np.swapaxes(v.values, 1, 2)
ug = np.swapaxes(ug.values, 1, 2)
vg = np.swapaxes(vg.values, 1, 2)
uu0 = np.swapaxes(np.squeeze(u0.values), 0, 1)  # swapaxes to (time,lat,lon)

lon=ds['lon'].values
lat=ds['lat'].values
time=ds['time']

# fill nan values to 0
uu[np.isnan(uu)] = 0
vv[np.isnan(vv)] = 0
ug[np.isnan(ug)] = 0
vg[np.isnan(vg)] = 0

ww = compute_ww(uu, vv, lat)
wg = compute_ww(ug, vg, lat)
    
coast = {'lat': [], 'lon': []}
    
for i, lat_val in enumerate(lat):
    for j, lon_val in enumerate(lon):            
        if np.isnan(uu0[i, j]):
            coast['lat'].append(lat_val.item())
            coast['lon'].append(lon_val.item())
            break  

n = 16  # coast band + ocean band
uc = np.empty((uu.shape[0],len(coast['lat']), n))
vc = np.empty((vv.shape[0],len(coast['lat']), n))
wc = np.empty((ww.shape[0],len(coast['lat']), n))
wgc = np.empty((wg.shape[0],len(coast['lat']), n))

lono = np.empty((len(coast['lat']), n))
lato = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    uc[:, point_idx, :] = uu[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    vc[:, point_idx, :] = vv[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    wc[:, point_idx, :] = ww[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    wgc[:, point_idx, :] = wg[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    lono[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    lato[point_idx, :] = lat[lat_idx]

#uc[np.abs(uc) > 100] = np.nan
#vc[np.abs(vc) > 100] = np.nan

# alongshore wind
def calculate_bearing(lat1, lon1, lat2, lon2):
    lat1_rad = np.deg2rad(lat1)
    lat2_rad = np.deg2rad(lat2)
    lon1_rad = np.deg2rad(lon1)
    lon2_rad = np.deg2rad(lon2)
    
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    
    x = delta_lon * np.cos((lat1_rad + lat2_rad) / 2)
    y = delta_lat
    
    bearing_rad = np.arctan2(y, x)
    bearing_deg = np.rad2deg(bearing_rad)
    bearing_deg = (bearing_deg + 360) % 360
    
    return bearing_deg

point_num = len(coast['lat'])
coast_orientation = np.empty(point_num)

for i in range(point_num):
    if i < point_num - 1:
        lat1 = coast['lat'][i]
        lon1 = coast['lon'][i]
        lat2 = coast['lat'][i+1]
        lon2 = coast['lon'][i+1]
    else:
        lat1 = coast['lat'][i-1]
        lon1 = coast['lon'][i-1]
        lat2 = coast['lat'][i]
        lon2 = coast['lon'][i]
    
    bearing = calculate_bearing(lat1, lon1, lat2, lon2)
    coast_orientation[i] = bearing

coast_orientation_rad = np.deg2rad(coast_orientation)
u_alongshore = np.cos(coast_orientation_rad).reshape(1, point_num, 1)
v_alongshore = np.sin(coast_orientation_rad).reshape(1, point_num, 1)
current_alongshore = uc * u_alongshore + vc * v_alongshore
np.savez(f'oscar_nointerp_{tmin}_{tmax}.npz', current_alongshore=current_alongshore, uc=uc, vc=vc, wc=wc, wgc=wgc)


## hflux

In [ ]:
# hfxc
import glob

data_dir = os.path.expandvars('$WORK/data/Tropflux/monthly')
file_pattern = os.path.join(data_dir, 'netflux_tropflux_1m_????.nc')
file_list = sorted(glob.glob(file_pattern))
ds = xr.open_mfdataset(file_list, combine='by_coords')
netflux=ds['netflux'].sel(longitude=slice(xmin,xmax), latitude=slice(ymin-1,ymax),time=slice(tmin,tmax))
netflux=netflux.interpolate_na(dim="longitude", method="nearest")
netflux=netflux.interpolate_na(dim="latitude", method="nearest")
netflux=netflux.bfill(dim="longitude")
netflux=netflux.bfill(dim="latitude")
lat_xr = xr.DataArray(lat, dims="latitude")
lon_xr = xr.DataArray(lon, dims="longitude")
hflx = netflux.interp(latitude=lat_xr, longitude=lon_xr).values

n = 16
hflxc = np.empty((hflx.shape[0],len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    hflxc[:, point_idx, :] = hflx[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
hflxu=hflxc[:,:,8:]-hflxc[:,:,:8]
np.savez(f'hflx_{tmin}_2018.npz', hflxc=hflxc, hflxu=hflxu, coast=coast, lonc=lonc, latc=latc, time=time)

In [ ]:
# swr
import glob

data_dir = os.path.expandvars('$WORK/data/Tropflux/monthly')
file_pattern = os.path.join(data_dir, 'swr_tropflux_1m_????.nc')
file_list = sorted(glob.glob(file_pattern))
ds = xr.open_mfdataset(file_list, combine='by_coords')
netflux=ds['swr'].sel(longitude=slice(xmin,xmax), latitude=slice(ymin-2,ymax),time=slice(tmin,tmax))
netflux=netflux.interpolate_na(dim="longitude", method="nearest")
netflux=netflux.interpolate_na(dim="latitude", method="nearest")
netflux=netflux.bfill(dim="longitude")
netflux=netflux.bfill(dim="latitude")
lat_xr = xr.DataArray(lat, dims="latitude")
lon_xr = xr.DataArray(lon, dims="longitude")
hflx = netflux.interp(latitude=lat_xr, longitude=lon_xr).values

n = 16
hflxc = np.empty((hflx.shape[0],len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    hflxc[:, point_idx, :] = hflx[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
hflxu=hflxc[:,:,8:]-hflxc[:,:,:8]
np.savez(f'swr_{tmin}_2018.npz', swrc=hflxc, hflxu=hflxu, coast=coast, lonc=lonc, latc=latc, time=time)

# fesom-core2-fjra

## heat budget

In [ ]:
# heat_budget core2-fjra
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-budget/analysis/fesom/fesom_core2_fjra_budget_mld1_m_1980_2023_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))

advx=ds['heat_advx'].groupby('time.month').mean(dim='time').values
advy=ds['heat_advy'].groupby('time.month').mean(dim='time').values
advz=ds['heat_advz'].groupby('time.month').mean(dim='time').values
diffv=ds['heat_diffv'].groupby('time.month').mean(dim='time').values
sbc=ds['heat_sbc'].groupby('time.month').mean(dim='time').values
tdc=ds['heat_tdc'].groupby('time.month').mean(dim='time').values

# also calculate the std and save it
advx_std=ds['heat_advx'].groupby('time.month').std(dim='time').values
advy_std=ds['heat_advy'].groupby('time.month').std(dim='time').values
advz_std=ds['heat_advz'].groupby('time.month').std(dim='time').values
diffv_std=ds['heat_diffv'].groupby('time.month').std(dim='time').values
sbc_std=ds['heat_sbc'].groupby('time.month').std(dim='time').values
tdc_std=ds['heat_tdc'].groupby('time.month').std(dim='time').values

n = 16  # coast band + ocean band

advx_c = np.empty((advx.shape[0], len(coast['lat']), n))
advy_c = np.empty((advy.shape[0], len(coast['lat']), n))
advz_c = np.empty((advz.shape[0], len(coast['lat']), n))
diffv_c = np.empty((diffv.shape[0], len(coast['lat']), n))
sbc_c = np.empty((sbc.shape[0], len(coast['lat']), n))
tdc_c = np.empty((tdc.shape[0], len(coast['lat']), n))

advx_c_std = np.empty((advx_std.shape[0], len(coast['lat']), n))
advy_c_std = np.empty((advy_std.shape[0], len(coast['lat']), n))
advz_c_std = np.empty((advz_std.shape[0], len(coast['lat']), n))
diffv_c_std = np.empty((diffv_std.shape[0], len(coast['lat']), n))
sbc_c_std = np.empty((sbc_std.shape[0], len(coast['lat']), n))
tdc_c_std = np.empty((tdc_std.shape[0], len(coast['lat']), n))

lono = np.empty((len(coast['lat']), n))
lato = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    advx_c[:, point_idx, :] = advx[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advy_c[:, point_idx, :] = advy[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advz_c[:, point_idx, :] = advz[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    diffv_c[:, point_idx, :] = diffv[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    sbc_c[:, point_idx, :] = sbc[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    tdc_c[:, point_idx, :] = tdc[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advx_c_std[:, point_idx, :] = advx_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advy_c_std[:, point_idx, :] = advy_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advz_c_std[:, point_idx, :] = advz_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    diffv_c_std[:, point_idx, :] = diffv_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    sbc_c_std[:, point_idx, :] = sbc_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    tdc_c_std[:, point_idx, :] = tdc_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    lono[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    lato[point_idx, :] = lat[lat_idx]

np.savez(f'heat_budget_mld1_fjra_{tmin}_{tmax}.npz',
         advx_c=advx_c, advy_c=advy_c, advz_c=advz_c,
         diffv_c=diffv_c, sbc_c=sbc_c, tdc_c=tdc_c,
         advx_c_std=advx_c_std, advy_c_std=advy_c_std, advz_c_std=advz_c_std,
         diffv_c_std=diffv_c_std, sbc_c_std=sbc_c_std, tdc_c_std=tdc_c_std,
         lono=lono, lato=lato)


## sst

In [ ]:
# core2-fjra
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-budget/analysis/fesom/fesom_core2_fjra_oce2d_m_1980_2023_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))
# calculate climological mean of sst
sst=ds['sst'].groupby('time.month').mean(dim='time').values
sst=ds['sst'].values

n = 16
sstc = np.empty((sst.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    sstc[:, point_idx, :] = sst[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
sstu=sstc[:,:,8:]-sstc[:,:,:8]
np.savez(f'fesom_core2_fjra_sstu_m_{tmin}_{tmax}.npz', lon=lon, lat=lat, sstu=sstu, sstc=sstc, coast=coast, lonc=lonc, latc=latc, time=time)

## wind

In [ ]:
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-budget/analysis/fesom/fesom_core2_fjra_forcing_m_1980_2023_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))

n = 8
u = ds['tx_sur'].values
v = ds['ty_sur'].values

uc = np.empty((u.shape[0],len(coast['lat']), n))
vc = np.empty((v.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    uc[:, point_idx, :] = u[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    vc[:, point_idx, :] = v[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]

# alongshore wind
def calculate_bearing(lat1, lon1, lat2, lon2):
    lat1_rad = np.deg2rad(lat1)
    lat2_rad = np.deg2rad(lat2)
    lon1_rad = np.deg2rad(lon1)
    lon2_rad = np.deg2rad(lon2)
    
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    
    x = delta_lon * np.cos((lat1_rad + lat2_rad) / 2)
    y = delta_lat
    
    bearing_rad = np.arctan2(y, x)
    bearing_deg = np.rad2deg(bearing_rad)
    bearing_deg = (bearing_deg + 360) % 360
    
    return bearing_deg

point_num = len(coast['lat'])
coast_orientation = np.empty(point_num)

for i in range(point_num):
    if i < point_num - 1:
        lat1 = coast['lat'][i]
        lon1 = coast['lon'][i]
        lat2 = coast['lat'][i+1]
        lon2 = coast['lon'][i+1]
    else:
        lat1 = coast['lat'][i-1]
        lon1 = coast['lon'][i-1]
        lat2 = coast['lat'][i]
        lon2 = coast['lon'][i]
    
    bearing = calculate_bearing(lat1, lon1, lat2, lon2)
    coast_orientation[i] = bearing

coast_orientation_rad = np.deg2rad(coast_orientation)
u_alongshore = np.cos(coast_orientation_rad).reshape(1, point_num, 1)
v_alongshore = np.sin(coast_orientation_rad).reshape(1, point_num, 1)
wind_alongshore = uc * u_alongshore + vc * v_alongshore

# Define the haversine function
coast_lon = np.array(coast['lon'])  # Shape: (100,)
coast_lat = np.array(coast['lat'])  # Shape: (100,)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  

    # Convert degrees to radians
    lat1_rad = np.deg2rad(lat1)
    lon1_rad = np.deg2rad(lon1)
    lat2_rad = np.deg2rad(lat2)
    lon2_rad = np.deg2rad(lon2)

    # Compute differences
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    # Haversine formula
    a = np.sin(delta_lat / 2.0) ** 2 + \
        np.cos(lat1_rad) * np.cos(lat2_rad) * \
        np.sin(delta_lon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

# Expand coast_lon and coast_lat for broadcasting
coast_lon_expanded = coast_lon[:, np.newaxis]  # Shape: (100, 1)
coast_lat_expanded = coast_lat[:, np.newaxis]  # Shape: (100, 1)

# Compute distances
distances = haversine(coast_lat_expanded, coast_lon_expanded, latc, lonc)  # Shape: (100, 8)

# Constants
rhoa = 1.22  # air density (kg/m^3)
cd = 0.0013  # drag coefficient
rho = 1025  # water density (kg/m^3)
omega = 7.2921e-5  # Earth's rotation rate (rad/s)

f = 2 * omega * np.sin(latc* np.pi / 180)

# Calculate magnitude of horizontal velocity
windspeed = np.sqrt(uc**2 + vc**2)

# Ekman transport calculation
etransport = -wind_alongshore / (rho * f * distances)
# wind_alongshore_mean = wind_alongshore[:, :, 0]
# mwind_alongshore_mean_filled = np.broadcast_to(wind_alongshore_mean[:, :, np.newaxis], wind_alongshore.shape)
# etransport = -wind_alongshore_mean_filled / (rho * f * distances)

epumping = np.zeros_like(uc)  # Ensure it has the right shape and is mutable
dy = 111000.0 * (lat[1] - lat[0])  # scalar
dv_dx = np.zeros_like(uc)
for j in range(lat.shape[0]):
    x_j = 111000.0 * np.cos(np.radians(lat[j])) * (lon[1] - lon[0]) 
    epumping[:,j,:] = 1 / (rho ) * (np.gradient(vc[:, j, :]/ f[j], x_j, axis=-1) - np.gradient(uc[:,j,:]/ f[j], dy, axis=1))

np.savez(f'fesom_core2_fjra_ekman_m_{tmin}_{tmax}.npz', windspeed=windspeed, etransport=etransport, epumping=epumping, coast=coast, lonc=lonc, latc=latc, time=time)

## heat flux

In [ ]:
# core2-fjra
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-budget/analysis/fesom/fesom_core2_fjra_forcing_m_1980_2023_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))
# calculate climological mean of sst
fh=-ds['fh'].groupby('time.month').mean(dim='time').values
n = 16
fhc = np.empty((fh.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    fhc[:, point_idx, :] = fh[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
fh=fhc[:,:,8:]-fhc[:,:,:8]
np.savez(f'fesom_core2_fjra_fh_m_{tmin}_{tmax}.npz', fh=fh)

## w

In [ ]:
# core2-fjra
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-budget/analysis/fesom/fesom_core2_fjra_w_mld1_m_1980_2023_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))
# calculate climological mean of sst
w=ds['w_mld'].groupby('time.month').mean(dim='time').values
n = 8
wc = np.empty((w.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    wc[:, point_idx, :] = w[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
np.savez(f'fesom_core2_fjra_w_mld1_m_{tmin}_{tmax}.npz', wc=wc)

## mld

In [ ]:
# core2-fjra
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-budget/analysis/fesom/fesom_core2_fjra_oce2d_m_1980_2023_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))
mld=-ds['MLD1'].groupby('time.month').mean(dim='time').values



n = 16
mldc = np.empty((mld.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    mldc[:, point_idx, :] = mld[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
np.savez(f'fesom_core2_fjra_mld1_m_{tmin}_{tmax}.npz', mldc=mldc)

## nflx_mld1

In [ ]:
# core2-fjra
ds_sbc = xr.open_mfdataset('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-budget/analysis/fesom/fesom_core2_fjra_budget_mld1_m_1980_2023_0.25.nc')
ds_sbc['lon'] = (ds_sbc['lon'] + 360) % 360
ds_sbc = ds_sbc.sortby('lon').sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))

ds_mld = xr.open_mfdataset('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-budget/analysis/fesom/fesom_core2_fjra_oce2d_m_1980_2023_0.25.nc')
ds_mld['lon'] = (ds_mld['lon'] + 360) % 360
ds_mld = ds_mld.sortby('lon').sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))

# nflx FIRST (keeps time)
nflx = ds_sbc['heat_sbc'] * (-ds_mld['MLD1']) * 4.2e6

nflux= nflx.groupby('time.month').mean(dim='time').values
n = 16
nfluxc = np.empty((nflux.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    nfluxc[:, point_idx, :] = nflux[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
np.savez(f'fesom_core2_fjra_nflux_mld1_m_{tmin}_{tmax}.npz', nfluxc=nfluxc)

# fesom-core2-fera5

## heat budget

In [ ]:
# heat_budget core2-fjra
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-fera5-budget/analysis/fesom/fesom_core2_fera5_budget_mld1_m_1980_2022_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))

advx=ds['heat_advx'].groupby('time.month').mean(dim='time').values
advy=ds['heat_advy'].groupby('time.month').mean(dim='time').values
advz=ds['heat_advz'].groupby('time.month').mean(dim='time').values
diffv=ds['heat_diffv'].groupby('time.month').mean(dim='time').values
sbc=ds['heat_sbc'].groupby('time.month').mean(dim='time').values
tdc=ds['heat_tdc'].groupby('time.month').mean(dim='time').values

# also calculate the std and save it
advx_std=ds['heat_advx'].groupby('time.month').std(dim='time').values
advy_std=ds['heat_advy'].groupby('time.month').std(dim='time').values
advz_std=ds['heat_advz'].groupby('time.month').std(dim='time').values
diffv_std=ds['heat_diffv'].groupby('time.month').std(dim='time').values
sbc_std=ds['heat_sbc'].groupby('time.month').std(dim='time').values
tdc_std=ds['heat_tdc'].groupby('time.month').std(dim='time').values

n = 16  # coast band + ocean band

advx_c = np.empty((advx.shape[0], len(coast['lat']), n))
advy_c = np.empty((advy.shape[0], len(coast['lat']), n))
advz_c = np.empty((advz.shape[0], len(coast['lat']), n))
diffv_c = np.empty((diffv.shape[0], len(coast['lat']), n))
sbc_c = np.empty((sbc.shape[0], len(coast['lat']), n))
tdc_c = np.empty((tdc.shape[0], len(coast['lat']), n))

advx_c_std = np.empty((advx_std.shape[0], len(coast['lat']), n))
advy_c_std = np.empty((advy_std.shape[0], len(coast['lat']), n))
advz_c_std = np.empty((advz_std.shape[0], len(coast['lat']), n))
diffv_c_std = np.empty((diffv_std.shape[0], len(coast['lat']), n))
sbc_c_std = np.empty((sbc_std.shape[0], len(coast['lat']), n))
tdc_c_std = np.empty((tdc_std.shape[0], len(coast['lat']), n))

lono = np.empty((len(coast['lat']), n))
lato = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    advx_c[:, point_idx, :] = advx[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advy_c[:, point_idx, :] = advy[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advz_c[:, point_idx, :] = advz[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    diffv_c[:, point_idx, :] = diffv[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    sbc_c[:, point_idx, :] = sbc[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    tdc_c[:, point_idx, :] = tdc[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advx_c_std[:, point_idx, :] = advx_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advy_c_std[:, point_idx, :] = advy_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advz_c_std[:, point_idx, :] = advz_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    diffv_c_std[:, point_idx, :] = diffv_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    sbc_c_std[:, point_idx, :] = sbc_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    tdc_c_std[:, point_idx, :] = tdc_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    lono[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    lato[point_idx, :] = lat[lat_idx]

np.savez(f'heat_budget_mld1_fera5_{tmin}_{tmax}.npz',
         advx_c=advx_c, advy_c=advy_c, advz_c=advz_c,
         diffv_c=diffv_c, sbc_c=sbc_c, tdc_c=tdc_c,
         advx_c_std=advx_c_std, advy_c_std=advy_c_std, advz_c_std=advz_c_std,
         diffv_c_std=diffv_c_std, sbc_c_std=sbc_c_std, tdc_c_std=tdc_c_std,
         lono=lono, lato=lato)


## sst

In [ ]:
# core2-fjra
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-fera5-budget/analysis/fesom/fesom_core2_fera5_oce2d_m_1980_2022_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))
# calculate climological mean of sst
sst=ds['sst'].groupby('time.month').mean(dim='time').values
n = 16
sstc = np.empty((sst.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    sstc[:, point_idx, :] = sst[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
sstu=sstc[:,:,8:]-sstc[:,:,:8]
np.savez(f'fesom_core2_fera5_sstu_m_{tmin}_{tmax}.npz', lon=lon, lat=lat, sstu=sstu, sstc=sstc, coast=coast, lonc=lonc, latc=latc, time=time)

## wind

In [ ]:
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-fera5-budget/analysis/fesom/fesom_core2_fera5_forcing_m_1980_2022_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))

n = 8
u = ds['tx_sur'].groupby('time.month').mean(dim='time').values
v = ds['ty_sur'].groupby('time.month').mean(dim='time').values
curl = ds['curl_surf'].groupby('time.month').mean(dim='time').values

uc = np.empty((u.shape[0],len(coast['lat']), n))
vc = np.empty((v.shape[0],len(coast['lat']), n))
curlc = np.empty((curl.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    uc[:, point_idx, :] = u[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    vc[:, point_idx, :] = v[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    curlc[:, point_idx, :] = curl[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]

# alongshore wind
def calculate_bearing(lat1, lon1, lat2, lon2):
    lat1_rad = np.deg2rad(lat1)
    lat2_rad = np.deg2rad(lat2)
    lon1_rad = np.deg2rad(lon1)
    lon2_rad = np.deg2rad(lon2)
    
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    
    x = delta_lon * np.cos((lat1_rad + lat2_rad) / 2)
    y = delta_lat
    
    bearing_rad = np.arctan2(y, x)
    bearing_deg = np.rad2deg(bearing_rad)
    bearing_deg = (bearing_deg + 360) % 360
    
    return bearing_deg

point_num = len(coast['lat'])
coast_orientation = np.empty(point_num)

for i in range(point_num):
    if i < point_num - 1:
        lat1 = coast['lat'][i]
        lon1 = coast['lon'][i]
        lat2 = coast['lat'][i+1]
        lon2 = coast['lon'][i+1]
    else:
        lat1 = coast['lat'][i-1]
        lon1 = coast['lon'][i-1]
        lat2 = coast['lat'][i]
        lon2 = coast['lon'][i]
    
    bearing = calculate_bearing(lat1, lon1, lat2, lon2)
    coast_orientation[i] = bearing

coast_orientation_rad = np.deg2rad(coast_orientation)
u_alongshore = np.cos(coast_orientation_rad).reshape(1, point_num, 1)
v_alongshore = np.sin(coast_orientation_rad).reshape(1, point_num, 1)
wind_alongshore = uc * u_alongshore + vc * v_alongshore

# Define the haversine function
coast_lon = np.array(coast['lon'])  # Shape: (100,)
coast_lat = np.array(coast['lat'])  # Shape: (100,)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  

    # Convert degrees to radians
    lat1_rad = np.deg2rad(lat1)
    lon1_rad = np.deg2rad(lon1)
    lat2_rad = np.deg2rad(lat2)
    lon2_rad = np.deg2rad(lon2)

    # Compute differences
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    # Haversine formula
    a = np.sin(delta_lat / 2.0) ** 2 + \
        np.cos(lat1_rad) * np.cos(lat2_rad) * \
        np.sin(delta_lon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

# Expand coast_lon and coast_lat for broadcasting
coast_lon_expanded = coast_lon[:, np.newaxis]  # Shape: (100, 1)
coast_lat_expanded = coast_lat[:, np.newaxis]  # Shape: (100, 1)

# Compute distances
distances = haversine(coast_lat_expanded, coast_lon_expanded, latc, lonc)  # Shape: (100, 8)

# Constants
rhoa = 1.22  # air density (kg/m^3)
cd = 0.0013  # drag coefficient
rho = 1025  # water density (kg/m^3)
omega = 7.2921e-5  # Earth's rotation rate (rad/s)

f = 2 * omega * np.sin(latc* np.pi / 180)

# Calculate magnitude of horizontal velocity
windspeed = np.sqrt(uc**2 + vc**2)

# Ekman transport calculation
etransport = -wind_alongshore / (rho * f * distances)
# wind_alongshore_mean = wind_alongshore[:, :, 0]
# mwind_alongshore_mean_filled = np.broadcast_to(wind_alongshore_mean[:, :, np.newaxis], wind_alongshore.shape)
# etransport = -wind_alongshore_mean_filled / (rho * f * distances)

epumping = np.zeros_like(uc)  # Ensure it has the right shape and is mutable
dy = 111000.0 * (lat[1] - lat[0])  # scalar
dv_dx = np.zeros_like(uc)
for j in range(lat.shape[0]):
    x_j = 111000.0 * np.cos(np.radians(lat[j])) * (lon[1] - lon[0]) 
    epumping[:,j,:] = 1 / (rho ) * (np.gradient(vc[:, j, :]/ f[j], x_j, axis=-1) - np.gradient(uc[:,j,:]/ f[j], dy, axis=1))

np.savez(f'fesom_core2_fera5_ekman_m_{tmin}_{tmax}.npz', windspeed=windspeed, etransport=etransport, epumping=epumping, coast=coast, lonc=lonc, latc=latc, time=time)

## heat flux

In [ ]:
# core2-fjra
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-fera5-budget/analysis/fesom/fesom_core2_fera5_forcing_m_1980_2022_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))
# calculate climological mean of sst
fh=-ds['fh'].groupby('time.month').mean(dim='time').values
n = 16
fhc = np.empty((fh.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    fhc[:, point_idx, :] = fh[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
fh=fhc[:,:,8:]-fhc[:,:,:8]
np.savez(f'fesom_core2_fera5_fh_m_{tmin}_{tmax}.npz', fh=fh)

## w_mld

In [ ]:
# core2-fjra
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-fera5-budget/analysis/fesom/fesom_core2_fera5_w_mld1_m_1980_2022_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))
# calculate climological mean of sst
w=ds['w_mld'].groupby('time.month').mean(dim='time').values
n = 8
wc = np.empty((w.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    wc[:, point_idx, :] = w[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
np.savez(f'fesom_core2_fera5_w_mld1_m_{tmin}_{tmax}.npz', wc=wc)

# fesom-core2-fcore2

In [ ]:
# heat_budget core2-fjra
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-core2-budget/analysis/fesom/fesom_core2_fcore2_budget_mld1_m_1980_2009_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))

advx=ds['heat_advx'].groupby('time.month').mean(dim='time').values
advy=ds['heat_advy'].groupby('time.month').mean(dim='time').values
advz=ds['heat_advz'].groupby('time.month').mean(dim='time').values
diffv=ds['heat_diffv'].groupby('time.month').mean(dim='time').values
sbc=ds['heat_sbc'].groupby('time.month').mean(dim='time').values
tdc=ds['heat_tdc'].groupby('time.month').mean(dim='time').values

# also calculate the std and save it
advx_std=ds['heat_advx'].groupby('time.month').std(dim='time').values
advy_std=ds['heat_advy'].groupby('time.month').std(dim='time').values
advz_std=ds['heat_advz'].groupby('time.month').std(dim='time').values
diffv_std=ds['heat_diffv'].groupby('time.month').std(dim='time').values
sbc_std=ds['heat_sbc'].groupby('time.month').std(dim='time').values
tdc_std=ds['heat_tdc'].groupby('time.month').std(dim='time').values

n = 16  # coast band + ocean band

advx_c = np.empty((advx.shape[0], len(coast['lat']), n))
advy_c = np.empty((advy.shape[0], len(coast['lat']), n))
advz_c = np.empty((advz.shape[0], len(coast['lat']), n))
diffv_c = np.empty((diffv.shape[0], len(coast['lat']), n))
sbc_c = np.empty((sbc.shape[0], len(coast['lat']), n))
tdc_c = np.empty((tdc.shape[0], len(coast['lat']), n))

advx_c_std = np.empty((advx_std.shape[0], len(coast['lat']), n))
advy_c_std = np.empty((advy_std.shape[0], len(coast['lat']), n))
advz_c_std = np.empty((advz_std.shape[0], len(coast['lat']), n))
diffv_c_std = np.empty((diffv_std.shape[0], len(coast['lat']), n))
sbc_c_std = np.empty((sbc_std.shape[0], len(coast['lat']), n))
tdc_c_std = np.empty((tdc_std.shape[0], len(coast['lat']), n))

lono = np.empty((len(coast['lat']), n))
lato = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    advx_c[:, point_idx, :] = advx[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advy_c[:, point_idx, :] = advy[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advz_c[:, point_idx, :] = advz[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    diffv_c[:, point_idx, :] = diffv[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    sbc_c[:, point_idx, :] = sbc[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    tdc_c[:, point_idx, :] = tdc[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advx_c_std[:, point_idx, :] = advx_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advy_c_std[:, point_idx, :] = advy_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advz_c_std[:, point_idx, :] = advz_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    diffv_c_std[:, point_idx, :] = diffv_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    sbc_c_std[:, point_idx, :] = sbc_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    tdc_c_std[:, point_idx, :] = tdc_std[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    lono[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    lato[point_idx, :] = lat[lat_idx]

np.savez(f'heat_budget_mld1_fcore2_{tmin}_2009.npz',
         advx_c=advx_c, advy_c=advy_c, advz_c=advz_c,
         diffv_c=diffv_c, sbc_c=sbc_c, tdc_c=tdc_c,
         advx_c_std=advx_c_std, advy_c_std=advy_c_std, advz_c_std=advz_c_std,
         diffv_c_std=diffv_c_std, sbc_c_std=sbc_c_std, tdc_c_std=tdc_c_std,
         lono=lono, lato=lato)


In [ ]:
advx_c.shape

In [ ]:
# core2-fcore2
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-core2-budget/analysis/fesom/fesom_core2_fcore2_oce2d_m_1980_2009_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))
# calculate climological mean of sst
sst=ds['sst'].groupby('time.month').mean(dim='time').values
n = 16
sstc = np.empty((sst.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    sstc[:, point_idx, :] = sst[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
sstu=sstc[:,:,8:]-sstc[:,:,:8]
np.savez(f'fesom_core2_fcore2_sstu_{tmin}_{tmax}.npz', lon=lon, lat=lat, sstu=sstu, sstc=sstc, coast=coast, lonc=lonc, latc=latc, time=time)

In [ ]:
# core2-fera5
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/core2-fera5-budget/analysis/fesom/fesom_core2_fera5_oce2d_m_1980_2022_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))
# calculate climological mean of sst
sst=ds['sst'].groupby('time.month').mean(dim='time').values
n = 16
sstc = np.empty((sst.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    sstc[:, point_idx, :] = sst[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
sstu=sstc[:,:,8:]-sstc[:,:,:8]
np.savez(f'fesom_core2_fera5_sstu_{tmin}_{tmax}.npz', lon=lon, lat=lat, sstu=sstu, sstc=sstc, coast=coast, lonc=lonc, latc=latc, time=time)

In [ ]:
# PERU
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
import cartopy.crs as ccrs
from matplotlib import cm

[xmin,xmax,ymin,ymax]=[270,290,-16,-6]
[tmin,tmax]=["1993","2022"] 
        
# heat_budget core2-fjra
file_path = os.path.expandvars('/gxfs_home/geomar/smomw639/FESOM/analysis/heat_budget_seasonal.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')

lon=ds['lon'].sel(lon=slice(xmin, xmax)).values
lat=ds['lat'].sel(lat=slice(ymin,ymax)).values
time=ds['time'].sel(time=slice(tmin,tmax))

sst0=ds['advx'].sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax)).isel(time=0).values
coast = {'lat': [], 'lon': []}
    
for i, lat_val in enumerate(lat):
    for j, lon_val in enumerate(lon):            
        if np.isnan(sst0[i, j]):
            coast['lat'].append(lat_val.item())
            coast['lon'].append(lon_val.item())
            break  

advx=ds['advx'].sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax))
advy=ds['advy'].sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax))
advz=ds['advz'].sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax))
diffv=ds['diffv'].sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax))
sbc=ds['sbc'].sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax))
tdc=ds['tdc'].sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax))

n = 24  # coast band + ocean band

advx_c = np.empty((advx.shape[0], len(coast['lat']), n))
advy_c = np.empty((advy.shape[0], len(coast['lat']), n))
advz_c = np.empty((advz.shape[0], len(coast['lat']), n))
diffv_c = np.empty((diffv.shape[0], len(coast['lat']), n))
sbc_c = np.empty((sbc.shape[0], len(coast['lat']), n))
tdc_c = np.empty((tdc.shape[0], len(coast['lat']), n))

lono = np.empty((len(coast['lat']), n))
lato = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    advx_c[:, point_idx, :] = advx[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advy_c[:, point_idx, :] = advy[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    advz_c[:, point_idx, :] = advz[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    diffv_c[:, point_idx, :] = diffv[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    sbc_c[:, point_idx, :] = sbc[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    tdc_c[:, point_idx, :] = tdc[:, lat_idx, int(lon_idx-n) : int(lon_idx)]
    lono[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    lato[point_idx, :] = lat[lat_idx]

np.savez(f'heat_budget_peru_fjra_{tmin}_{tmax}.npz',
         advx_c=advx_c, advy_c=advy_c, advz_c=advz_c,
         diffv_c=diffv_c, sbc_c=sbc_c, tdc_c=tdc_c,
         lono=lono, lato=lato)


# markus

In [ ]:
# core2-fjra
file_path = os.path.expandvars('/gxfs_work/geomar/smomw639/runtime/fesom-2.6/core2/markus/analysis/fesom/fesom_core2_fjra_oce2d_m_1980_2023_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))
# calculate climological mean of sst
sst=ds['sst'].groupby('time.month').mean(dim='time').values
n = 16
sstc = np.empty((sst.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    sstc[:, point_idx, :] = sst[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
sstu=sstc[:,:,8:]-sstc[:,:,:8]
np.savez(f'fesom_core2_markus_sstu_m_{tmin}_{tmax}.npz', lon=lon, lat=lat, sstu=sstu, sstc=sstc, coast=coast, lonc=lonc, latc=latc, time=time)

# fesom-tropotest-fjra

In [ ]:
file_path = os.path.expandvars('/gxfs_home/geomar/smomw639/Paper1/fesom_tropotest_fjra_oce2d_m_1993_2009_0.25.nc')
ds = xr.open_dataset(
    file_path
)
ds['lon'] = (ds['lon'] + 360) % 360
ds=ds.sortby('lon')
ds=ds.sel(lon=slice(xmin,xmax), lat=slice(ymin,ymax), time=slice(tmin,tmax))
# calculate climological mean of sst
sst=ds['sst'].groupby('time.month').mean(dim='time').values
n = 16
sstc = np.empty((sst.shape[0],len(coast['lat']), n))
lonc = np.empty((len(coast['lat']), n))
latc = np.empty((len(coast['lat']), n))

for point_idx, (lat_val, lon_val) in enumerate(zip(coast['lat'], coast['lon'])):
    lat_idx = np.abs(lat - lat_val).argmin()
    lon_idx = np.abs(lon - lon_val).argmin()
    sstc[:, point_idx, :] = sst[:, lat_idx, int(lon_idx-n) : int(lon_idx)]      # costal band
    lonc[point_idx, :] = lon[int(lon_idx-n) : int(lon_idx)]
    latc[point_idx, :] = lat[lat_idx]
sstu=sstc[:,:,8:]-sstc[:,:,:8]
np.savez(f'fesom_tropotest_fjra_sstu_m_{tmin}_{tmax}.npz', lon=lon, lat=lat, sstu=sstu, sstc=sstc, coast=coast, lonc=lonc, latc=latc, time=time)